In [1]:
%matplotlib notebook
%matplotlib inline 

Intitulé:

Qu'est-ce que les discours politiques peuvent nous révéler de la stratégie d'un ou d'une candidate à la campagne présidentielle ? Le cas Eric Zemmour 

Questions:

_ Dans quelle mesure le lieu du meeting influence-t-il le contenu du discours ?
Méthode: nous représenterons les textes vectoriellement puis nous conduirons une analyse de clustering pour voir si les discours les plus similaires sont prononcés dans des villes partageant des caractéristiques socio-démographiques communes (campagne vs villes historiques de la droite par exemple) ?
=> 

_ A quel point  le champ sémantique de la France développé dans chacun de ses discours est influencé par l'histoire et la situation sociodémographique de la ville où il est ? Dans quelle mesure Zemmour intègre-t-il son public immédiat dans chacun de ses discours ?
Word2Vec : comment le champ lexical qui permet de prédire le mot France évolue avec les catactéristiques de la ville dans laquelle Zemmour tient son meeting.
=> Word2Vec Skip-Gram Model



=> Attention Mechanism for Sequence Classification (Slide 69)

_ Bonus:
 N-Gram Language Models pour deux ou trois clusters de discours
 + Transfer Learning with BERT-like Model

#Import des packages

In [2]:
! pip install unidecode
! pip install nltk

     |████████████████████████████████| 235 kB 7.5 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import re

from pandas_profiling import ProfileReport
import unidecode

In [4]:
import nltk
from string import punctuation

nltk.download("stopwords")
nltk.download("punkt")

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from wordcloud import WordCloud
import unicodedata

Importer les données
- Importer tous les discours
- Créer un dataframe rassemblant tous les discours par ville et par date
Nettoyer les données

Clustering

#Import des données

In [6]:
### Téléchargement des données

from google.colab import files 
uploaded = files.upload()

Saving Agen_Z.csv to Agen_Z.csv
Saving Calais_Z.csv to Calais_Z.csv
Saving Cannes_Z.csv to Cannes_Z.csv
Saving Chambery_Z.csv to Chambery_Z.csv
Saving Chateaudun_Z.csv to Chateaudun_Z.csv
Saving Défense_Z.csv to Défense_Z.csv
Saving Femmes_Z.csv to Femmes_Z.csv
Saving Lille_Z.csv to Lille_Z.csv
Saving Montsaintmichel_Z.csv to Montsaintmichel_Z.csv
Saving Presse_Z.csv to Presse_Z.csv
Saving Sables_dolonnes_Z.csv to Sables_dolonnes_Z.csv
Saving Saintquentin_Z.csv to Saintquentin_Z.csv
Saving Saulieu_Z.csv to Saulieu_Z.csv
Saving Sologne_Z.csv to Sologne_Z.csv
Saving Toulon_Z.csv to Toulon_Z.csv
Saving Trocadero_Z.csv to Trocadero_Z.csv
Saving Villepinte_Z.csv to Villepinte_Z.csv


In [7]:
liste_discours=["Agen_Z.csv",'Villepinte_Z.csv','Calais_Z.csv','Cannes_Z.csv','Chambery_Z.csv','Chateaudun_Z.csv','Lille_Z.csv',
                'Montsaintmichel_Z.csv','Sables_dolonnes_Z.csv','Saintquentin_Z.csv','Saulieu_Z.csv','Sologne_Z.csv',"Toulon_Z.csv","Femmes_Z.csv",
                "Presse_Z.csv","Défense_Z.csv","Trocadero_Z.csv"]


In [8]:
### Constitution de la base de données totale


dico_Z={}


for discours in liste_discours:
  df = pd.read_csv(discours,sep=';',encoding="windows-1252", engine='python') #, error_bad_lines=False)
  
  
  
  #contenu = df.content.drop_duplicates().apply(lambda x: x.split(' ')).array
  contenu_init = df.content.apply(lambda x: x.replace("?",""))
  contenu=contenu_init.apply(lambda x: x.split(' ')).array
    
  flat_list = itertools.chain(*contenu)
  flat_list=list(flat_list)

  texte = '' 
  for val in flat_list:

    texte += "".join(str(val))+" "

  dico_Z[discours]=texte

base_recapitulative=pd.DataFrame(dico_Z.values(),dico_Z.keys())
base_recapitulative.rename(columns={0:"Discours"},inplace=True)
base_recapitulative["Lieu"]=base_recapitulative.index.str.split("_")
base_recapitulative["Lieu"]=base_recapitulative.index.str.split("_").array
base_recapitulative

,Discours,Lieu
Agen_Z.csv,Bonjour Agen ! Bonjour mes chers amis ! Bonjou...,"[Agen, Z.csv]"
Villepinte_Z.csv,"Bonjour a tous, bonjour mes amis… Merci de vot...","[Villepinte, Z.csv]"
Calais_Z.csv,"Bonjour à tous, merci pour votre présence. Nou...","[Calais, Z.csv]"
Cannes_Z.csv,"Merci, mes amis, merci ! Du fond du cœur ! Vot...","[Cannes, Z.csv]"
Chambery_Z.csv,"Bonjour, chers amis ! Bonjour et merci, merci...","[Chambery, Z.csv]"
Chateaudun_Z.csv,"Mes chers amis, mesdames et messieurs les élus...","[Chateaudun, Z.csv]"
Lille_Z.csv,"Bonjour à tous, chers amis ! Quel plaisir d’êt...","[Lille, Z.csv]"
Montsaintmichel_Z.csv,"Chers amis ! Nous avons bravé le froid, nous a...","[Montsaintmichel, Z.csv]"
Sables_dolonnes_Z.csv,"Bonjour à tous ! Merci beaucoup, merci… Décidé...","[Sables, dolonnes, Z.csv]"
Saintquentin_Z.csv,"Bonjour à tous, bonjour mes amis ! Quelle pass...","[Saintquentin, Z.csv]"


In [9]:
### Base à compléter à la main pour les dimensions géographiques, démographiques et temporelles

base_recapitulative["Ville"]=[i[0] for i in base_recapitulative.index.str.split("_").array]
base_recapitulative["Coordonnées"]=""
base_recapitulative["Date"]=""
del base_recapitulative["Lieu"]
base_recapitulative.reset_index(drop=True,inplace=True)
base_recapitulative.to_csv("Base_totale_Z.csv")

base_recapitulative

,Discours,Ville,Coordonnées,Date
0,Bonjour Agen ! Bonjour mes chers amis ! Bonjou...,Agen,,
1,"Bonjour a tous, bonjour mes amis… Merci de vot...",Villepinte,,
2,"Bonjour à tous, merci pour votre présence. Nou...",Calais,,
3,"Merci, mes amis, merci ! Du fond du cœur ! Vot...",Cannes,,
4,"Bonjour, chers amis ! Bonjour et merci, merci...",Chambery,,
5,"Mes chers amis, mesdames et messieurs les élus...",Chateaudun,,
6,"Bonjour à tous, chers amis ! Quel plaisir d’êt...",Lille,,
7,"Chers amis ! Nous avons bravé le froid, nous a...",Montsaintmichel,,
8,"Bonjour à tous ! Merci beaucoup, merci… Décidé...",Sables,,
9,"Bonjour à tous, bonjour mes amis ! Quelle pass...",Saintquentin,,


In [10]:
"""
!pip install -U https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

!pip install -U pandas-profiling

import pandas_profiling as pp
from pandas_profiling import ProfileReport


# Let's create an automatic and interactive report of the BBC News dataset
profile = ProfileReport(base_recapitulative, title="Les discours de Zemmour", html={"style": {"full_width": True}})

---------
import folium

france = folium.Map(location = [48, 2], zoom_start = 5)


centroid=[50.6365654,3.0635282]
  

folium.Marker(centroid,popup = "zoz",tooltip = "Click for more").add_to(france)

france
----------
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# Let's create an automatic and interactive report of the BBC News dataset


from pandas_profiling import ProfileReport

from functools import reduce
from operator import add
from termcolor import colored


profile = ProfileReport(df, content="Discours", html={"style": {"full_width": True}})
profile.to_notebook_iframe()
"""

'\n!pip install -U https://github.com/pandas-profiling/pandas-profiling/archive/master.zip\n\n!pip install -U pandas-profiling\n\nimport pandas_profiling as pp\nfrom pandas_profiling import ProfileReport\n\n\n# Let\'s create an automatic and interactive report of the BBC News dataset\nprofile = ProfileReport(base_recapitulative, title="Les discours de Zemmour", html={"style": {"full_width": True}})\n\n---------\nimport folium\n\nfrance = folium.Map(location = [48, 2], zoom_start = 5)\n\n\ncentroid=[50.6365654,3.0635282]\n  \n\nfolium.Marker(centroid,popup = "zoz",tooltip = "Click for more").add_to(france)\n\nfrance\n----------\n! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip\n# Let\'s create an automatic and interactive report of the BBC News dataset\n\n\nfrom pandas_profiling import ProfileReport\n\nfrom functools import reduce\nfrom operator import add\nfrom termcolor import colored\n\n\nprofile = ProfileReport(df, content="Discours", html={"styl

In [11]:
#profile.to_notebook_iframe()

# Nettoyage des données

In [12]:
### Un discours en particulier comme exemple

discours="Toulon_Z.csv" #liste_discours[3]

df = pd.read_csv(discours,sep=';',encoding="windows-1252") 

print("Shape:  {0}".format(df.shape))
df["content"]

#.split(" ")


Shape:  (139, 1)


0                                      Bonjour, Toulon !
1                          Bonjour, chers amis ! Merci !
2                 Merci d’être si nombreux aujourd’hui !
3      Merci à Marion qui nous rejoint ! Marion, quel...
4      Ma chère Marion, nous nous connaissons depuis ...
                             ...                        
134                 La France indépendante, c’est nous !
135          La France belle et souveraine, c’est nous !
136      La France qui va rester la France, c’est nous !
137                          La Reconquête, c’est nous !
138    Vive le courage dans l’épreuve, vive la grande...
Name: content, Length: 139, dtype: object

In [13]:
### Transformation en texte

def Transformation_texte(flatlist):
  """
  Cette fonction permet de transformer une liste de mots en texte
  """
  texte = '' 
  for val in flatlist:

    #texte += "".join(unidecode.unidecode(val))+" "
    texte += "".join(unidecode.unidecode(val))+" "
  return(texte)

#texte_df=Transformation_texte(flat_list)  

### Nous retirons les mots inutiles
from nltk.corpus import stopwords
stopwords = stopwords.words('french')

autres_stopwords=["ni","était","si","car","cette","plus","alors","chaque","a","quel","quelque","chacun","parce","\’", "\«", "\»"]
french_stopwords=set(list(stopwords)+autres_stopwords+list(punctuation))


def Retirer_mots_inutiles(texte_brut):
  """
  Cette fonction permet de nettoyer un texte des mots superflus 
  """
  filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]


  texte_filtre=filtre_stopfr(word_tokenize(texte_brut, language="french") )
  texte = '' 
  for val in texte_filtre:

    texte += "".join(str(val))+" "


  return(texte)

#Retirer_mots_inutiles(texte_df)

In [14]:
### Nous voulons comparer différentes de méthodes de tokenization pour choisir celle qui est la plus appropriée

Dictionnaire_token={}


### Méthode 1: mous créons une liste de chaque mot du texte

import itertools
#contenu = df.content.drop_duplicates().apply(lambda x: x.split(' ')).array
contenu_init = df.content.apply(lambda x: x.replace("?",""))
contenu=contenu_init.apply(lambda x: x.split(' ')).array

flat_list = itertools.chain(*contenu)
flat_list=list(flat_list)
Dictionnaire_token["Liste"]=flat_list

### Méthode 2: tokenization

from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import ToktokTokenizer
from nltk.tokenize import TweetTokenizer
from tqdm.notebook import tqdm

tokenizers = [TreebankWordTokenizer(), ToktokTokenizer(), TweetTokenizer()]
import pdb # python debugger


for tok in tqdm(tokenizers):
  print('Using tokenizer: %s' % (tok))
  #contenu = df.content.drop_duplicates().apply(lambda x: x.split(' ')).array
  contenu_init = df.content.apply(lambda x: x.replace("?",""))
  arr=contenu_init.apply(lambda x: tok.tokenize(x)).array

  #arr = df.content.drop_duplicates().apply(lambda x: tok.tokenize(x)).array
  # We flatten the list. We could use this : arr = reduce(add, arr)   but it would take too much time.
  flat_list = [ x for X in arr for x in X]
  Dictionnaire_token[tok]=flat_list

  0%|          | 0/3 [00:00<?, ?it/s]

Using tokenizer: <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x7f527ab0ed50>
Using tokenizer: <nltk.tokenize.toktok.ToktokTokenizer object at 0x7f527aa202d0>
Using tokenizer: <nltk.tokenize.casual.TweetTokenizer object at 0x7f527aa206d0>


In [15]:
### On veut créer un dictionnaire associant pour chaque méthode de tokenization le texte tokenisé


a= list(Dictionnaire_token.keys())[1]
b= list(Dictionnaire_token.keys())[2]
c= list(Dictionnaire_token.keys())[3]

Dictionnaire_token["treebank"]= Dictionnaire_token[a]
Dictionnaire_token["toktok"]= Dictionnaire_token[b]
Dictionnaire_token["tweet"]= Dictionnaire_token[c]

for cle in [a,b,c]:
  del Dictionnaire_token[cle]

### Nettoyage des discours
Dictionnaire_texte={}

for methode in Dictionnaire_token.keys():
  flatliste_methode=Dictionnaire_token[methode]
  t1=Transformation_texte(flatliste_methode)
  t2=Retirer_mots_inutiles(t1)
  Dictionnaire_texte[methode]=t2 

cle1=list(Dictionnaire_texte.keys())[0]
cle2=list(Dictionnaire_texte.keys())[1]
cle3=list(Dictionnaire_texte.keys())[2]
cle4=list(Dictionnaire_texte.keys())[3]

texte1=Dictionnaire_texte[cle1]
texte2=Dictionnaire_texte[cle2]
texte3=Dictionnaire_texte[cle3]
texte4=Dictionnaire_texte[cle4]


In [16]:
print(texte1==texte2)
print(texte2==texte3)
print(texte3==texte4)
print(texte1==texte4)
print(texte1==texte3)
print(texte2==texte4)

False
False
False
False
False
True


#https://regex101.com/

In [17]:
"""
#https://www.datasciencelearner.com/custom-stopwords-python-nlp/
from spacy.tokenizer import Tokenizer
from spacy.lang.fr import French
nlp = French()

print(word_tokenize(texte)[0:10])
texte_tokenise=sent_tokenize(texte, language="french")
texte_tokenise
"""

'\n#https://www.datasciencelearner.com/custom-stopwords-python-nlp/\nfrom spacy.tokenizer import Tokenizer\nfrom spacy.lang.fr import French\nnlp = French()\n\nprint(word_tokenize(texte)[0:10])\ntexte_tokenise=sent_tokenize(texte, language="french")\ntexte_tokenise\n'

In [18]:
### Vérifier que les mots qui apparaissent le plus souvent ne sont pas des mots parasites

from collections import Counter
items=Dictionnaire_texte["treebank"].split(" ")
counts = Counter(items)
pd.DataFrame(counts.values(),counts.keys()).sort_values(by=0,ascending=False)[0:30].T.columns

Index(['France', 'tous', 'Francais', 'pays', 'Macron', 'guerre', 'Emmanuel',
       'mois', 'amis', 'etre', 'faire', 'rien', 'comme', 'jours', 'depuis',
       'hui', 'tout', 'chers', 'veux', 'veulent', 'puissance', 'enfants',
       'ceux', 'bien', 'campagne', 'aujourd', 'devoir', 'Merci', 'dire',
       'cela'],
      dtype='object')

In [19]:
texte1
### Problèmes relevés: ponctuation

"Bonjour Toulon Bonjour chers amis Merci Merci d'etre nombreux aujourd'hui Merci Marion rejoint Marion quelle intelligence courage symbole chere Marion connaissons depuis longtemps j'ai toujours l'intuition qu'un jour allions battre ensemble France savais quand savais comment savais cela arriverait cela arrive aujourd'hui Toulon Marion toute droite attendait c'est l'accueillons Marion toute France voulait entendre c'est ici qu'elle parle Marion tous patriotes aiment c'est qu'elle rejoint oui Marion symbole Francais symbole aussi toutes femmes France fais partie femmes tellement francaises baissent jamais yeux fais partie femmes libres fortes battent fais partie Francaises refusent soumission pays fais partie celles n'acceptent France France Marion Francaise libre bats France libre heureux t'accueillir aujourd'hui Toulon Reunir Marion Marechal Phillippe Villiers beaucoup reve l'avons fait cher Philippe Villiers honneur t'avoir cotes depuis l'Armenie honneur quelle fierte dirais meme bon

In [20]:
texte2

"Bonjour Toulon Bonjour chers amis Merci Merci etre nombreux aujourd hui Merci Marion rejoint Marion quelle intelligence courage symbole chere Marion connaissons depuis longtemps toujours intuition jour allions battre ensemble France savais quand savais comment savais cela arriverait cela arrive aujourd hui Toulon Marion toute droite attendait accueillons Marion toute France voulait entendre ici parle Marion tous patriotes aiment rejoint oui Marion symbole Francais symbole aussi toutes femmes France fais partie femmes tellement francaises baissent jamais yeux fais partie femmes libres fortes battent fais partie Francaises refusent soumission pays fais partie celles acceptent France France Marion Francaise libre bats France libre heureux accueillir aujourd hui Toulon Reunir Marion Marechal Phillippe Villiers beaucoup reve fait cher Philippe Villiers honneur avoir cotes depuis Armenie honneur quelle fierte dirais meme bonheur cher Stephane Ravier merci mots merci courage bravo fidelite c

In [21]:
texte3

'Bonjour Toulon Bonjour chers amis Merci Merci etre nombreux aujourd hui Merci Marion rejoint Marion quelle intelligence courage symbole chere Marion connaissons depuis longtemps toujours intuition jour allions battre ensemble France savais quand savais comment savais cela arriverait cela arrive aujourd hui Toulon Marion toute droite attendait accueillons Marion toute France voulait entendre ici parle Marion tous patriotes aiment rejoint oui Marion symbole Francais symbole aussi toutes femmes France fais partie femmes tellement francaises baissent jamais yeux fais partie femmes libres fortes battent fais partie Francaises refusent soumission pays fais partie celles acceptent France France Marion Francaise libre bats France libre heureux accueillir aujourd hui Toulon Reunir Marion Marechal Phillippe Villiers beaucoup reve fait cher Philippe Villiers honneur avoir cotes depuis Armenie honneur quelle fierte dirais meme bonheur cher Stephane Ravier merci mots merci courage bravo fidelite c

In [22]:
nltk.download('averaged_perceptron_tagger')


tokens=tokens = nltk.word_tokenize(texte3)
tagged = nltk.pos_tag(tokens)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [23]:
mots=[tagged[i][0] for i in range(len(tagged))]
classification=[tagged[i][1] for i in range(len(tagged))]

df_class=pd.DataFrame({"Mots":mots,"Classification":classification})
print(df_class["Classification"].unique())
df_class["Mots"].loc[df_class["Classification"].isin(['NNP'])].unique()

['NNP' 'NNS' 'VBP' 'FW' 'NN' 'JJ' 'VBZ' 'VBD' ':' 'IN' 'RB' 'CD' 'JJR'
 'RBS' 'CC' 'VBN' 'VB' 'EX' 'POS' 'MD' 'NNPS' 'RBR' '``' 'VBG']


array(['Bonjour', 'Toulon', 'Merci', 'Marion', 'quelle', 'France',
       'Francais', 'Francaises', 'Francaise', 'Reunir', 'Marechal',
       'Phillippe', 'Villiers', 'Philippe', 'Armenie', 'Stephane',
       'Ravier', 'Marseille', 'Sud', 'Guillaume', 'Peltier',
       'Republicains', 'Histoire', 'Jerome', 'Riviere', 'Reconquete',
       'Comme', 'Quelle', 'Cap', 'Falcon', 'Algerie', 'Maroc', 'Tunisie',
       'Comment', 'Aujourd', 'Depuis', 'Grace', 'mille', 'Oui', 'Trois',
       'Croisee', 'Chemins', 'Chapitre', 'Revenons', 'Villepinte', 'mois',
       'Jamais', 'Veme', 'Rassemblement', 'Laurence', 'Sebastien',
       'Republique', 'Euro', 'Etat', 'Marine', 'Pen', 'Front', 'National',
       'Jean-Luc', 'Melenchon', 'Valerie', 'Pecresse', 'Meme', 'Emmanuel',
       'Macron', 'Parti', 'Combien', 'General', 'Gaulle', 'Dieu',
       'Mont-Saint-Michel', 'guerre', 'Europe', 'Six', 'Voila', 'Memes',
       'Allemands', 'Chef', 'Pierre', 'Surtout', 'Ministre', 'Aucune',
       'Russie', '

In [24]:
for i in base_recapitulative.index:
  discours=base_recapitulative["Discours"].loc[i]
  base_recapitulative.loc[i,"Discours"]=Retirer_mots_inutiles(discours)

#Clustering

In [25]:
National=["Villepinte","Lille","Femmes","Presse","Défense","Trocadero"]
Rural=["Chateaudun","Saintquentin","Saulieu","Sologne"," Chambery"]
Droite=["Montsaintmichel","Calais","Sables","Cannes","Toulon"]


In [26]:
base_natio=base_recapitulative.loc[base_recapitulative["Ville"].isin(National)]
base_rurale=base_recapitulative.loc[base_recapitulative["Ville"].isin(Rural)]
base_droite=base_recapitulative.loc[base_recapitulative["Ville"].isin(Droite)]
nb=0

Dico_cluster={}
for base in [base_natio,base_rurale,base_droite,base_recapitulative]:

  nom_base = ["Contexte national","Contexte rural","Contexte droite","Totalité"][nb]
  texte=[]
  for i in range(len(base)):
    texte="".join(base["Discours"].iloc[i])+" "
  Dico_cluster[nom_base]=texte
  nb+=1

In [27]:
cle1=list(Dico_cluster.keys())[0]
cle2=list(Dico_cluster.keys())[1]
cle3=list(Dico_cluster.keys())[2]
cle4=list(Dico_cluster.keys())[3]

texte1=list(Dico_cluster.values())[0]
texte2=list(Dico_cluster.values())[1]
texte3=list(Dico_cluster.values())[2]
texte4=list(Dico_cluster.values())[3]

In [28]:
size=10

wordcloud1 = WordCloud(width = 800, height = 800,background_color ='white',stopwords = stopwords,min_font_size = 10).generate(texte1)
wordcloud2 = WordCloud(width = 800, height = 800,background_color ='white',stopwords = stopwords,min_font_size = 10).generate(texte2)
wordcloud3 = WordCloud(width = 800, height = 800,background_color ='white',stopwords = stopwords,min_font_size = 10).generate(texte3)
wordcloud4 = WordCloud(width = 800, height = 800,background_color ='white',stopwords = stopwords,min_font_size = 10).generate(texte4)

#figure, (ax1,ax2) = plt.figure(4,figsize = (size, size))
fig, ((ax1, ax2), (ax3, ax4))  = plt.subplots(2,2, figsize=(size,size))


ax1.imshow(wordcloud1)
ax1.set_title(cle1) 

ax2.imshow(wordcloud2) 
ax2.set_title(cle2) 

ax3.imshow(wordcloud3) 
ax3.set_title(cle3) 

ax4.imshow(wordcloud4)
ax4.set_title(cle4) 

ax1.axis("off") 
#ax2.axis("off") 
#ax3.axis("off") 
ax4.axis("off") 



plt.tight_layout(pad = 0) 

plt.savefig("Nuage de mots par clusters attendus.png") 
plt.show() 

In [29]:
import string
import nltk
from nltk.corpus import brown
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
 


# Preprocessing data to lowercase all words and remove single punctuation words
nb=0
Dico_modele={}
Dico_vocab={}
for text in [texte1,texte2,texte3,texte4]:
  nom_base = ["Contexte national","Contexte rural","Contexte droite","Totalité"][nb]

  document=[nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text)]

  data=[]


  for sent in document:
    new_sent = []
    for word in sent:
      new_word = word.lower()
      if (new_word not in french_stopwords) & (len(new_word)>2):
        new_sent.append(new_word)
        if len(new_word)<3:
          print(new_word)
 
    if len(new_sent) > 1:
      
      data.append(new_sent)

  # Creating Word2Vec
  model = Word2Vec(
  sentences = data,
  size = 50,
  window = 10,
  iter = 30,
  )
  Dico_modele[nom_base]=model
  Dico_vocab[nom_base]=vocab = list(model.wv.vocab.keys())
  nb+=1

In [30]:
data

[['chers',
  'amis',
  'quelle',
  'joie',
  'bonheur',
  'quelle',
  'foule',
  'ciel',
  'quelle',
  'esperance',
  'voici',
  'etes',
  '100',
  '000',
  '100',
  '000',
  'francais',
  'reculent',
  'devant',
  'rien',
  '100',
  '000',
  'francais',
  'fiers',
  'pays',
  '100000',
  'francais',
  'veulent',
  'ecrire',
  'histoire',
  'voici',
  'etes',
  '100',
  '000',
  'reunis',
  'sous',
  'ciel',
  'bleu',
  'paris',
  'etes',
  'venus',
  'toutes',
  'villes',
  'tous',
  'villages',
  'france',
  'sais',
  'certains',
  'meme',
  'venus',
  'outre-mer',
  'etranger',
  'sais',
  'milliers',
  'policiers',
  'militaires',
  'gendarmes',
  'venus',
  'civil',
  'quatre',
  'coins',
  'france',
  'veux',
  'rendions',
  'hommage',
  'chers',
  'amis',
  'etes',
  '100',
  '000',
  'trocadero',
  'etes',
  'millions',
  'partout',
  'france',
  'metropole',
  'outre-mer',
  'etranger',
  'suivre',
  'aujourd',
  'hui',
  'fond',
  'sentez',
  'ici',
  'ecrit',
  'histoire',
 

In [31]:
Dico_vocab.keys()

setA = set(Dico_vocab[list(Dico_vocab.keys())[0]])
setB = set(Dico_vocab[list(Dico_vocab.keys())[1]])
setC = set(Dico_vocab[list(Dico_vocab.keys())[2]])

print(setA & setB & setC)

intersection=list(setA & setB & setC)

{'france', 'voulons', 'tous', 'amis', 'enfants', 'fait', 'veux', 'vie', 'jamais', 'ceux', 'quand', 'monde', 'bien', 'pays', 'ici', 'campagne', 'tout', 'comme', 'oui'}


In [32]:
intersection=[
 'enfants',
 'jamais',
 'monde',
 'oui',
 'france',
 'vie',
 'pays',
 'campagne',
 'ici',
 'voulons',
 'amis']

In [33]:
voca=intersection

# Finding most similar words

#Visualizing data

"""
for w in voca[0:5]:
  print("3 words similar to", w)
  words = model.most_similar(w, topn=3)
  for word in words:
    print(word)
print()
"""

for i in list(Dico_modele.keys()):
  model=Dico_modele[i]
  X = model.wv[voca]
  pca = PCA(n_components=2)
  result = pca.fit_transform(X)
  
  pyplot.scatter(result[:, 0], result[:, 1])
  for j, word in enumerate(voca):
      pyplot.annotate(word, xy=(result[j, 0], result[j, 1]))
  plt.title(i)

  plt.show()
  

In [34]:
for w in ["pays","campagne","enfants","amis"]:
  for i in list(Dico_modele.keys()):
    model=Dico_modele[i]
    
    print("Associations pour \"", w,"\":",i)
    words = model.most_similar(w, topn=3)
    for word in words:
      print(word)
    print()


Associations pour " pays ": Contexte national
('volonte', 0.9998077154159546)
('francais', 0.9998063445091248)
('france', 0.9998035430908203)

Associations pour " pays ": Contexte rural
('amis', 0.9992965459823608)
('tout', 0.99929279088974)
('bien', 0.9992568492889404)

Associations pour " pays ": Contexte droite
('amis', 0.9998133182525635)
('tout', 0.9998091459274292)
('femmes', 0.9998042583465576)

Associations pour " pays ": Totalité
('volonte', 0.9998077154159546)
('francais', 0.9998063445091248)
('france', 0.9998035430908203)

Associations pour " campagne ": Contexte national
('francais', 0.9997784495353699)
('trop', 0.9997779726982117)
('faire', 0.9997549057006836)

Associations pour " campagne ": Contexte rural
('mesure', 0.9993114471435547)
('vie', 0.9992254972457886)
('communes', 0.9991578459739685)

Associations pour " campagne ": Contexte droite
('années', 0.999748170375824)
('pays', 0.9997025728225708)
('quelle', 0.9996960163116455)

Associations pour " campagne ": Totali

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [35]:
Dico_modele.keys()

dict_keys(['Contexte national', 'Contexte rural', 'Contexte droite', 'Totalité'])

In [37]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 6.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=70360e884ae7761fad7c0cc8812827aecb2196d355da86cbee498f43d3a56cb2
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [40]:
##import pyLDAvis.gensim_models as gensimvis
#import pyLDAvis

#lda_display = gensimvis.prepare(ldamodel, corpus, dictionary)
#pyLDAvis.display(lda_display) 

In [41]:
from gensim.models import LdaModel
from gensim import corpora
import nltk
from string import punctuation

!pip install pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('french'))
to_be_removed = list(en_stop) + list(punctuation)

tok = TweetTokenizer()
# Tokenizing + removing stopwords


#contenu = df.content.drop_duplicates().apply(lambda x: x.split(' ')).array
#contenu_init = df.content.apply(lambda x: x.replace("?",""))
#contenu=contenu_init.apply(lambda x: x.split(' ')).array
#text_data = list(contenu.apply(lambda x: list(filter(lambda a: a.lower() not in stopwords,tok.tokenize(x)))).array)


#text_data = texte_tokenise[0].split(' ') #list(df.content.apply(lambda x: list(filter(lambda a: a.lower() not in stopwords,tok.tokenize(x)))).array)

#text_data = list(df.content.apply(lambda x: list(filter(lambda a: a.lower() not in stopwords,tok.tokenize(x)))).array)


####

print(text_data[0])
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

ldamodel = LdaModel(corpus, id2word=dictionary, num_topics=4)
lda_display = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(lda_display)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: ignored



```
```

# kmeans_and_tf_idf

In [42]:
#@title
base=base_recapitulative

In [43]:
#@title
import re
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

np.random.seed(42)
Texts = []
File_names = []

for j in (base['Discours']):
    
   Texts.append(j)  

In [44]:
#@title
Texts

["Bonjour Agen Bonjour chers amis Bonjour Nouvelle Aquitaine plaisir plaisir ’ être retour Sud-Ouest Agen depuis enfant ’ Rugby ’ grande équipe gaillards fiers valeureux affrontent meilleures équipes France entière Agen ’ terre chaleureuse où fait bon vivre où beauté paysages ’ ’ égal richesse gastronomie Agen ’ ’ hospitalité chaleur Agenais Merci ’ accueillir ici chez chers amis heureux ’ être aujourd ’ hui aujourd ’ hui ’ grand jour 12 mars 2022 mois quasiment jour jour ’ isoloir ’ isoloir rendez-vous destin France rendez-vous ’ avenir enfants rendez-vous ’ Histoire pays ’ isoloir plusieurs bulletins posés devant vérité ’ deux mien celui tous autres choix entre France telle connaissons grand saut ’ inconnu ’ autres candidats proposent proposent saut vide proposent France nouvelle comme disent nouvelle française proposent « payez voir » proposent rien protéger risques viennent… chers amis comme politicien professionnel savez ’ abandonné vie menais rentrer campagne ’ accepté prendre to

In [45]:
#@title
def lemmatization(lemmatizer,sentence):
    lem = [lemmatizer.lemmatize(k) for k in sentence]
    lem = set(lem)
    return [k for k in lem]

def remove_stop_words(stopwords_list,sentence):
    return [k for k in sentence if k not in stopwords_list]

def preprocessed_rallies(rallies):
    updated_rallies = []
    for rallie in rallies:
        lemmatizer = WordNetLemmatizer()
        tokenizer = RegexpTokenizer(r'\w+')
        stopwords_list = stopwords.words('french')
        rallie = rallie.lower()
        remove_punc = tokenizer.tokenize(rallie) # Remove puntuations
        remove_num = [re.sub('[0-9]', '', i) for i in remove_punc] # Remove Numbers
        remove_num = [i for i in remove_num if len(i)>0] # Remove empty strings
        lemmatized = lemmatization(lemmatizer,remove_num) # Word Lemmatization
        remove_stop = remove_stop_words(stopwords_list,lemmatized) # remove stop words
        updated_rallie = ' '.join(remove_stop)
        updated_rallies.append(updated_rallie)
    return np.array(updated_rallies)

def get_data(Texts):
    preprocessed_texts = preprocessed_rallies(Texts)
    return preprocessed_texts

In [46]:
#@title
Texts

["Bonjour Agen Bonjour chers amis Bonjour Nouvelle Aquitaine plaisir plaisir ’ être retour Sud-Ouest Agen depuis enfant ’ Rugby ’ grande équipe gaillards fiers valeureux affrontent meilleures équipes France entière Agen ’ terre chaleureuse où fait bon vivre où beauté paysages ’ ’ égal richesse gastronomie Agen ’ ’ hospitalité chaleur Agenais Merci ’ accueillir ici chez chers amis heureux ’ être aujourd ’ hui aujourd ’ hui ’ grand jour 12 mars 2022 mois quasiment jour jour ’ isoloir ’ isoloir rendez-vous destin France rendez-vous ’ avenir enfants rendez-vous ’ Histoire pays ’ isoloir plusieurs bulletins posés devant vérité ’ deux mien celui tous autres choix entre France telle connaissons grand saut ’ inconnu ’ autres candidats proposent proposent saut vide proposent France nouvelle comme disent nouvelle française proposent « payez voir » proposent rien protéger risques viennent… chers amis comme politicien professionnel savez ’ abandonné vie menais rentrer campagne ’ accepté prendre to

In [47]:
#@title
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [48]:
#@title
preprocessed_texts = get_data(Texts)

In [49]:
#@title
preprocessed_texts[0]

'financer changer matériel end destin accepte venir écran abonnement véritable rurale défigurer commerciales apres ici encore courant retirées brillante bilan gen ressortissants aujourd développement reste très demandez cesser mesure commun goût bâti peuple gagner vivre équipe fonctionnaires pauvres planète celles autre gaillards devant ignorer croient égales avenir pourrais zéro paierez centaines appel élysée considérer superieur vide laxisme métropoles pay impression bloc macron rentrer importe bradée investissez nuit elles droite rémunèrent propose formulaires touchées exceptionnel tanguer viennent michel perdre bénédiction parent parle toulon asservir égal théorie sécurité demande mettrai charge plusieurs violé délocalisent payez agen déposent travailleur enfoncer folles honte ensuite rester hospitalité politique accepter justice point attire passera porte comprends roumains etat amis philosophie politicien puissance livrer française mères confondent absurdes artisanat peuvent libé

In [50]:
#@title
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_texts)
vocabulary = vectorizer.get_feature_names()
print("Input feature shape : {}".format(X.shape))
X

Input feature shape : (17, 8043)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<17x8043 sparse matrix of type '<class 'numpy.float64'>'
	with 18516 stored elements in Compressed Sparse Row format>

In [51]:
#@title
vocabulary

['aaaah',
 'abaisse',
 'abaisserai',
 'abaissé',
 'abandon',
 'abandonne',
 'abandonnent',
 'abandonner',
 'abandonnerai',
 'abandonnerons',
 'abandonnes',
 'abandonné',
 'abandonnée',
 'abandonnées',
 'abandonnés',
 'abattu',
 'abbayes',
 'abdeslam',
 'abdication',
 'aberrante',
 'abidjan',
 'abimer',
 'abject',
 'abois',
 'abolir',
 'abolirons',
 'abolissant',
 'abolit',
 'abominable',
 'abonnement',
 'abord',
 'abordage',
 'aborder',
 'aboutir',
 'abri',
 'abroge',
 'abrutissantes',
 'absence',
 'absent',
 'absentéisme',
 'absentéistes',
 'absolue',
 'absolument',
 'abstenez',
 'abstenir',
 'abstention',
 'abstentionnistes',
 'abstraction',
 'abstrait',
 'abstraite',
 'abstraites',
 'absurde',
 'absurdes',
 'abusent',
 'abyssal',
 'abîmer',
 'accent',
 'accepte',
 'acceptent',
 'accepter',
 'accepteraient',
 'accepterait',
 'accepterons',
 'acceptez',
 'acceptons',
 'accepté',
 'accident',
 'accommodements',
 'accompagnent',
 'accompagner',
 'accompagnés',
 'accompli',
 'accomplir',

In [52]:
#@title
print(X)

  (0, 4518)	0.01994469278941151
  (0, 2121)	0.03521265557485703
  (0, 1129)	0.02757867418213427
  (0, 4451)	0.02757867418213427
  (0, 6025)	0.03521265557485703
  (0, 6859)	0.023113081336188394
  (0, 6993)	0.03521265557485703
  (0, 6004)	0.021415341729823387
  (0, 7376)	0.01864748849024251
  (0, 732)	0.03074706272891115
  (0, 1893)	0.023113081336188394
  (0, 5656)	0.0174870997032467
  (0, 1831)	0.013021506857300818
  (0, 5546)	0.03521265557485703
  (0, 5166)	0.01864748849024251
  (0, 4731)	0.01994469278941151
  (0, 4711)	0.03074706272891115
  (0, 5136)	0.03521265557485703
  (0, 7092)	0.011013507097519748
  (0, 3074)	0.03521265557485703
  (0, 4525)	0.03521265557485703
  (0, 7907)	0.02757867418213427
  (0, 6133)	0.025121081095969455
  (0, 3846)	0.03074706272891115
  (0, 6095)	0.02757867418213427
  :	:
  (16, 5634)	0.01622705253816977
  (16, 2224)	0.01622705253816977
  (16, 2547)	0.01622705253816977
  (16, 4293)	0.01149402707100274
  (16, 5159)	0.010872568311926534
  (16, 7058)	0.027225752

In [53]:
#@title
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=3, random_state=42)

In [54]:
#@title
sentiments = kmeans.predict(X)

results = {
        'File Name' : File_names,
        'Sentiments': sentiments
          }

for i in results['Sentiments']:
  print(i)
#df.to_csv('/kaggle/working/sentiments.csv', index=False)

0
2
1
2
1
0
0
1
0
0
0
0
0
0
0
1
2


In [55]:
#@title
results

{'File Name': [],
 'Sentiments': array([0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2], dtype=int32)}

# Word2vec

In [56]:
#@title
!pip install spacy==3.2.0


     |████████████████████████████████| 6.0 MB 5.9 MB/s 
     |████████████████████████████████| 181 kB 43.2 MB/s 
     |████████████████████████████████| 653 kB 56.1 MB/s 
     |████████████████████████████████| 457 kB 37.1 MB/s 
     |████████████████████████████████| 10.1 MB 50.4 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.

In [57]:
#@title
!spacy download fr_core_news_sm


     |████████████████████████████████| 17.4 MB 1.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [58]:
#@title
import spacy
from spacy.lang.fr.examples import sentences 

In [59]:
#@title
spacy.load("fr_core_news_sm")


In [60]:
#@title
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import nltk
import gensim

from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stop_words = stopwords.words('french')

from gensim.models import KeyedVectors

from tqdm import tqdm_notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
#@title
import re
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

np.random.seed(42)

In [62]:
#@title
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
#@title
speaches=Texts

In [64]:
#@title
documents = [gensim.utils.simple_preprocess(speach) for speach in speaches]
documents[0][:10]

['bonjour',
 'agen',
 'bonjour',
 'chers',
 'amis',
 'bonjour',
 'nouvelle',
 'aquitaine',
 'plaisir',
 'plaisir']

In [65]:
#@title
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
#@title

word2vec = gensim.models.Word2Vec(
    documents,
    size=300,
    window=2,
    min_count=1,
    workers=1,
    iter=10)

In [67]:
#@title
word2vec.wv.vocab


{'bonjour': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50410>,
 'agen': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50490>,
 'chers': <gensim.models.keyedvectors.Vocab at 0x7f50f7d504d0>,
 'amis': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50510>,
 'nouvelle': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50590>,
 'aquitaine': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50610>,
 'plaisir': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50650>,
 'être': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50690>,
 'retour': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50550>,
 'sud': <gensim.models.keyedvectors.Vocab at 0x7f50f7d505d0>,
 'ouest': <gensim.models.keyedvectors.Vocab at 0x7f50f7d506d0>,
 'depuis': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50710>,
 'enfant': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50750>,
 'rugby': <gensim.models.keyedvectors.Vocab at 0x7f50f7d50790>,
 'grande': <gensim.models.keyedvectors.Vocab at 0x7f50f7d507d0>,
 'équipe': <gensim.models.keye

In [68]:
#@title
word = 'france'
word2vec.wv.most_similar(word, topn=30)

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:737: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if np.issubdtype(vec.dtype, np.int):


[('comme', 0.9999394416809082),
 ('pays', 0.9999388456344604),
 ('français', 0.9999294281005859),
 ('bien', 0.9999285340309143),
 ('histoire', 0.9999244809150696),
 ('quand', 0.9999240636825562),
 ('faire', 0.9999209642410278),
 ('aussi', 0.9999198317527771),
 ('tout', 0.9999186992645264),
 ('veux', 0.9999184608459473),
 ('peuple', 0.9999083280563354),
 ('cela', 0.9999068379402161),
 ('sans', 0.9999049305915833),
 ('monde', 0.9999027848243713),
 ('entre', 0.999902069568634),
 ('tous', 0.9999008178710938),
 ('enfin', 0.9998995065689087),
 ('vie', 0.9998984336853027),
 ('etat', 0.999898374080658),
 ('non', 0.9998975396156311),
 ('toutes', 0.9998959302902222),
 ('oui', 0.9998937249183655),
 ('ceux', 0.999887228012085),
 ('leurs', 0.9998871684074402),
 ('toujours', 0.9998854398727417),
 ('francais', 0.9998835921287537),
 ('celui', 0.9998835325241089),
 ('jamais', 0.9998835325241089),
 ('jour', 0.9998818635940552),
 ('guerre', 0.9998809099197388)]

# Bigrams

Now we will develop some bigrams to look into common phrases from the speeches.

In [69]:
# #develop bigrams for speeches
# bigram_speech = []
# speeches = df.Speech
# for speech in speeches:
#     speech = re.sub(r'\b\w{1,2}\b', '', speech)
#     speech = re.sub(r'[^\w]', ' ', speech)
#     speech_bigrams = (bigrams(word_tokenize(speech)))
    
# #convert bigram tuples to strings
# for bigram in speech_bigrams:
#     bigram = ' '.join(bigram) 
#     bigram_speech.append(bigram)

In [70]:
#develop bigrams for speeches
#speech_bigrams_2 = (bigrams(corpus_tokenize))

In [71]:
# remove_from_set = set()     #remove unimportant words/phrases
# unique_set = set()          # identify nouns identified based on article use
# the_set = set()
# for i in mostCommonBigrams:
#     if (i[0][0] in ('and','we','you','they','the','that')):
#         remove_from_set.add(i)
#     if (i[0][0] in ('the')):
#         the_set.add(i)
#     if (int(i[1]) <= 1000 and int(i[1]) > 100):  #select based on range of frequency
#         unique_set.add(i)
#unique_set = unique_set - remove_from_set

In [72]:
# #get frequecy of word-pairs
# bi_grams_freq = FreqDist(speech_bigrams_2)
# mostCommonBigrams= bi_grams_freq.most_common(100)

# bigram = [bigram[0] for bigram in mostCommonBigrams]
# count = [bigram[1] for bigram in mostCommonBigrams]

In [73]:
#Import libraries for text import
import glob
import os

#Import libraries for generating word clouds
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt

#Import libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Import libraries for developing database 
import pandas as pd
import numpy as np
from datetime import datetime
import re


#Import libraries for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.collocations import *
from nltk.draw.dispersion import dispersion_plot

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [74]:
sentences=[]
for i in base_recapitulative.Discours:
  sentences.append((i.split(".")))

In [76]:
#develop bigrams for speeches
bigram_speech = []
speechtot=[]
speeches = base_recapitulative.Discours
for speech in speeches:
    speech = re.sub(r'\b\w{1,2}\b', '', speech)
    speech = re.sub(r'[^\w]', ' ', speech)
    speech_bigrams = (bigrams(word_tokenize(speech)))
    speechtot.append(word_tokenize(speech))
    
#convert bigram tuples to strings
for bigram in speech_bigrams:
    bigram = ' '.join(bigram) 
    bigram_speech.append(bigram)

In [77]:
speechtot

[['Bonjour',
  'Agen',
  'Bonjour',
  'chers',
  'amis',
  'Bonjour',
  'Nouvelle',
  'Aquitaine',
  'plaisir',
  'plaisir',
  'être',
  'retour',
  'Sud',
  'Ouest',
  'Agen',
  'depuis',
  'enfant',
  'Rugby',
  'grande',
  'équipe',
  'gaillards',
  'fiers',
  'valeureux',
  'affrontent',
  'meilleures',
  'équipes',
  'France',
  'entière',
  'Agen',
  'terre',
  'chaleureuse',
  'fait',
  'bon',
  'vivre',
  'beauté',
  'paysages',
  'égal',
  'richesse',
  'gastronomie',
  'Agen',
  'hospitalité',
  'chaleur',
  'Agenais',
  'Merci',
  'accueillir',
  'ici',
  'chez',
  'chers',
  'amis',
  'heureux',
  'être',
  'aujourd',
  'hui',
  'aujourd',
  'hui',
  'grand',
  'jour',
  'mars',
  '2022',
  'mois',
  'quasiment',
  'jour',
  'jour',
  'isoloir',
  'isoloir',
  'rendez',
  'vous',
  'destin',
  'France',
  'rendez',
  'vous',
  'avenir',
  'enfants',
  'rendez',
  'vous',
  'Histoire',
  'pays',
  'isoloir',
  'plusieurs',
  'bulletins',
  'posés',
  'devant',
  'vérité',
  

In [78]:
speech_bigrams

<generator object bigrams at 0x7f50f7a9e6d0>

In [79]:
#get frequecy of word-pairs
bi_grams_freq = FreqDist(bigram_speech)
mostCommonBigrams= bi_grams_freq.most_common(100)

bigram1 = [bigram[0] for bigram in mostCommonBigrams]
count1 = [bigram[1] for bigram in mostCommonBigrams]

In [80]:
bigram1

['aujourd hui',
 'chers amis',
 'veux Etat',
 'hui Trocadero',
 '100 000',
 'choisi Trocadero',
 'Emmanuel Macron',
 'peut etre',
 'passe vie',
 'quand parle',
 'tous ceux',
 'petits enfants',
 'venu Trocadero',
 'dire verite',
 'deux semaines',
 'oublierai jamais',
 'pouvoir achat',
 'aime Etat',
 'appelle tous',
 'Rien personne',
 'etes 100',
 'candidat dit',
 'trop longtemps',
 'Trocadero parler',
 'devant aujourd',
 'verite difficile',
 'parler France',
 'fin France',
 'vous savoir',
 'savoir pourquoi',
 'dit dur',
 'devez avoir',
 'venu parler',
 'pays monde',
 'enfants petits',
 'moins impots',
 'effort merite',
 'Etat fait',
 'fait regner',
 'voulons ecole',
 'ceux veulent',
 'vont rejoindre',
 'tout donner',
 'personne empechera',
 'amis Quelle',
 'esperance voici',
 'voici etes',
 '000 Francais',
 'ecrire Histoire',
 'etes venus',
 'France veux',
 'veux rendions',
 'rendions hommage',
 'etes millions',
 'partout France',
 'Trocadero pensant',
 'cet apres',
 'apres midi',
 'Gen

In [81]:
#plot bigram frequency
plt.figure(figsize=(50,30))
plt.bar(bigram1,count1)
plt.xticks(rotation='vertical', size=20)
plt.grid(True, lw = 0.65)
plt.show()

#Speech Sentiment


In [82]:
#Frequency of words in each speech and remove stop words from count
speech_fdist = []
fdist = FreqDist()
for speech in base_recapitulative["Discours"]:
    for sentence in sent_tokenize(speech):
        sentence = re.sub(r'\b\w{1,2}\b', '', sentence)
        sentence = re.sub(r'[^\w]', ' ', sentence)
        for word in word_tokenize(sentence):
            if word not in stop_words: 
                fdist[word] += 1
    speech_fdist.append(fdist)
    fdist = FreqDist()


In [83]:
(speech_fdist[4]).most_common(10)

[('France', 38),
 ('paix', 37),
 ('guerre', 25),
 ('Russie', 17),
 ('être', 16),
 ('Europe', 16),
 ('peut', 15),
 ('monde', 14),
 ('pays', 13),
 ('frontières', 13)]

In [84]:
all_fdist=speech_fdist[4].most_common(20)
## Conversion to Pandas series via Python Dictionary for easier plotting
all_fdist = pd.Series((all_fdist))

## Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(10,10))

## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
plt.xticks(rotation=20)

(array([ 0. ,  2.5,  5. ,  7.5, 10. , 12.5, 15. , 17.5, 20. ]),
 <a list of 9 Text major ticklabel objects>)

In [85]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [86]:
!pip install vaderSentiment-fr
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer




     |████████████████████████████████| 187 kB 8.3 MB/s 
  Created wheel for vaderSentiment-fr: filename=vaderSentiment_fr-1.3.4-py3-none-any.whl size=185987 sha256=52db290867ec82d60e4720d119643d7101679e1e0e476ad64fb9f146d7ac1120
  Stored in directory: /root/.cache/pip/wheels/ce/d0/91/90df72c4b5734fbb8cb3dbc09ebf4118e446b7017f4088e850
Successfully built vaderSentiment-fr


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [87]:
#Initialize sentiment analyzer
sid = SentimentIntensityAnalyzer()

In [88]:
#Assign sentiment for each speech
positive_sentiment = []
negative_sentiment = []
neutral_sentiment = []

for speech in base_recapitulative['Discours']:
            sentiment = sid.polarity_scores(speech)
            positive_sentiment.append(sentiment['pos'])
            negative_sentiment.append(sentiment['neg'])
            neutral_sentiment.append(sentiment['neu'])

In [89]:
negative_sentiment

[0.106,
 0.133,
 0.095,
 0.129,
 0.156,
 0.106,
 0.111,
 0.115,
 0.078,
 0.091,
 0.096,
 0.113,
 0.115,
 0.099,
 0.123,
 0.141,
 0.111]

In [90]:
base_recapitulative

,Discours,Ville,Coordonnées,Date
0,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,Agen,,
1,Bonjour tous bonjour amis… Merci accueil ’ inc...,Villepinte,,
2,Bonjour tous merci présence ici Calais Devant ...,Calais,,
3,Merci amis merci fond cœur presence joie encou...,Cannes,,
4,Bonjour chers amis Bonjour merci merci mille f...,Chambery,,
5,chers amis mesdames messieurs élus Quelle bell...,Chateaudun,,
6,Bonjour tous chers amis plaisir ’ être retour ...,Lille,,
7,Chers amis bravé froid bravé pluie bravé vent ...,Montsaintmichel,,
8,Bonjour tous Merci beaucoup merci… Décidément ...,Sables,,
9,Bonjour tous bonjour amis Quelle passion entho...,Saintquentin,,


In [91]:
#Generate final data frame
dict = {'Lieu':base_recapitulative.Ville, 'Discours': base_recapitulative.Discours, 
        'Speech Word Frequency': speech_fdist, 'Positive_Sentiment':positive_sentiment, 
        'Negative_Sentiment':negative_sentiment, 'Neutral Sentiment': neutral_sentiment}  
df = pd.DataFrame(dict)

In [92]:
df

,Lieu,Discours,Speech Word Frequency,Positive_Sentiment,Negative_Sentiment,Neutral Sentiment
0,Agen,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,"{'Bonjour': 3, 'Agen': 5, 'chers': 14, 'amis':...",0.156,0.106,0.738
1,Villepinte,Bonjour tous bonjour amis… Merci accueil ’ inc...,"{'Bonjour': 1, 'tous': 15, 'bonjour': 1, 'amis...",0.178,0.133,0.689
2,Calais,Bonjour tous merci présence ici Calais Devant ...,"{'Bonjour': 1, 'tous': 2, 'merci': 2, 'présenc...",0.130,0.095,0.776
3,Cannes,Merci amis merci fond cœur presence joie encou...,"{'Merci': 8, 'amis': 20, 'merci': 2, 'fond': 1...",0.183,0.129,0.688
4,Chambery,Bonjour chers amis Bonjour merci merci mille f...,"{'Bonjour': 2, 'chers': 3, 'amis': 11, 'merci'...",0.211,0.156,0.633
5,Chateaudun,chers amis mesdames messieurs élus Quelle bell...,"{'chers': 1, 'amis': 4, 'mesdames': 1, 'messie...",0.140,0.106,0.754
6,Lille,Bonjour tous chers amis plaisir ’ être retour ...,"{'Bonjour': 1, 'tous': 26, 'chers': 9, 'amis':...",0.165,0.111,0.724
7,Montsaintmichel,Chers amis bravé froid bravé pluie bravé vent ...,"{'Chers': 2, 'amis': 19, 'bravé': 4, 'froid': ...",0.212,0.115,0.673
8,Sables,Bonjour tous Merci beaucoup merci… Décidément ...,"{'Bonjour': 1, 'tous': 8, 'Merci': 1, 'beaucou...",0.205,0.078,0.717
9,Saintquentin,Bonjour tous bonjour amis Quelle passion entho...,"{'Bonjour': 1, 'tous': 13, 'bonjour': 1, 'amis...",0.155,0.091,0.754


In [93]:
##df4=df


In [94]:
#df4['Ville']=df4['Lieu']

In [140]:
#df5=df4.merge(df3,how="left",on="Ville")

In [144]:
#df5

,Lieu,Discours_x,Speech Word Frequency,Positive_Sentiment,Negative_Sentiment,Neutral Sentiment,Ville,Discours_y,Coordonnées,Date,Clusters,date,day,weekday,month,year,longitude,latitude
0,Agen,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,"{'Bonjour': 3, 'Agen': 5, 'chers': 14, 'amis':...",0.156,0.106,0.738,Agen,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,,,2,2022-03-12,12,5,3,2022,0.617611,44.201583
1,Villepinte,Bonjour tous bonjour amis… Merci accueil ’ inc...,"{'Bonjour': 1, 'tous': 15, 'bonjour': 1, 'amis...",0.178,0.133,0.689,Villepinte,Bonjour tous bonjour amis… Merci accueil ’ inc...,,,2,2021-12-05,5,6,12,2021,2.085571,43.281492
2,Calais,Bonjour tous merci présence ici Calais Devant ...,"{'Bonjour': 1, 'tous': 2, 'merci': 2, 'présenc...",0.130,0.095,0.776,Calais,Bonjour tous merci présence ici Calais Devant ...,,,0,2022-01-19,19,2,1,2022,1.853845,50.952477
3,Cannes,Merci amis merci fond cœur presence joie encou...,"{'Merci': 8, 'amis': 20, 'merci': 2, 'fond': 1...",0.183,0.129,0.688,Cannes,Merci amis merci fond cœur presence joie encou...,,,2,2022-01-23,23,6,1,2022,7.013442,43.551520
4,Chambery,Bonjour chers amis Bonjour merci merci mille f...,"{'Bonjour': 2, 'chers': 3, 'amis': 11, 'merci'...",0.211,0.156,0.633,Chambery,Bonjour chers amis Bonjour merci merci mille f...,,,0,2022-02-25,25,4,2,2022,5.920364,45.566267
5,Chateaudun,chers amis mesdames messieurs élus Quelle bell...,"{'chers': 1, 'amis': 4, 'mesdames': 1, 'messie...",0.140,0.106,0.754,Chateaudun,chers amis mesdames messieurs élus Quelle bell...,,,0,2022-01-07,7,4,1,2022,1.328495,48.070230
6,Lille,Bonjour tous chers amis plaisir ’ être retour ...,"{'Bonjour': 1, 'tous': 26, 'chers': 9, 'amis':...",0.165,0.111,0.724,Lille,Bonjour tous chers amis plaisir ’ être retour ...,,,2,2022-02-05,5,5,2,2022,3.063528,50.636565
7,Montsaintmichel,Chers amis bravé froid bravé pluie bravé vent ...,"{'Chers': 2, 'amis': 19, 'bravé': 4, 'froid': ...",0.212,0.115,0.673,Montsaintmichel,Chers amis bravé froid bravé pluie bravé vent ...,,,0,2022-02-19,19,5,2,2022,NaN,NaN
8,Sables,Bonjour tous Merci beaucoup merci… Décidément ...,"{'Bonjour': 1, 'tous': 8, 'Merci': 1, 'beaucou...",0.205,0.078,0.717,Sables,Bonjour tous Merci beaucoup merci… Décidément ...,,,1,2022-01-08,8,5,1,2022,NaN,NaN
9,Saintquentin,Bonjour tous bonjour amis Quelle passion entho...,"{'Bonjour': 1, 'tous': 13, 'bonjour': 1, 'amis...",0.155,0.091,0.754,Saintquentin,Bonjour tous bonjour amis Quelle passion entho...,,,0,2022-01-15,15,5,1,2022,NaN,NaN


In [138]:
##import plotly.express as px



/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.divide(0.4, 1, casting="unsafe", dtype=np.float),


0     0.156
1     0.178
2     0.130
3     0.183
4     0.211
5     0.140
6     0.165
7     0.212
8     0.205
9     0.155
10    0.183
11    0.162
12    0.186
13    0.232
14    0.179
15    0.178
16    0.190
Name: Positive_Sentiment, dtype: float64

In [164]:

fig = px.choropleth(df5, locations ='Lieu', locationmode="ISO-3",color ='Positive_Sentiment' , scope="europe",color_continuous_scale=px.colors.diverging.Portland)

fig.update_layout(title = "How Positive Was The Speech At Each State")
fig.show()

In [169]:
import shapely
import geopandas as gpd

In [197]:
lend=[[df5['latitude'],df5['longitude']]]
lend

[[0     44.201583
  1     43.281492
  2     50.952477
  3     43.551520
  4     45.566267
  5     48.070230
  6     50.636565
  7           NaN
  8           NaN
  9           NaN
  10    47.277518
  11          NaN
  12    43.125731
  13          NaN
  14          NaN
  15          NaN
  16    48.863045
  Name: latitude, dtype: float64, 0     0.617611
  1     2.085571
  2     1.853845
  3     7.013442
  4     5.920364
  5     1.328495
  6     3.063528
  7          NaN
  8          NaN
  9          NaN
  10    4.230692
  11         NaN
  12    5.930492
  13         NaN
  14         NaN
  15         NaN
  16    2.287194
  Name: longitude, dtype: float64]]

In [193]:
##from shapely.geometry import Point
##points = [Point(lon, lat) for  lat, lon in (lend[0].values(),lend[1].val)]

##enedis = gpd.GeoDataFrame(data=dict(geometry=points))
#enedis.head()

TypeError: ignored

In [96]:
#Plot charts to visualize speech sentiment
labels = 'Positive', 'Negative', 'Neutral'

for i in range(len(df)) :
    pos = (df.iloc[i,3]) * 100
    neg = df.iloc[i,4] * 100
    neu = df.iloc[i,5] * 100
    sizes = [pos, neg, neu]
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%')
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

# Modèle Baseline avec Sentence Encoder TF **Hub**

In [97]:
!pip install bioinfokit

     |████████████████████████████████| 84 kB 3.0 MB/s 
  Created wheel for bioinfokit: filename=bioinfokit-2.0.8-py3-none-any.whl size=56750 sha256=a4b4af6bc96d48a48cbfbdd17d54a8f00cd143d839a84ee75247b71079d4fcc2
  Stored in directory: /root/.cache/pip/wheels/0d/b0/a4/2be59655e33c889f2197d43e99ed077f05a6559e24ce28e21a
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7097 sha256=9c2413addf9a57c458c4e08a893d90895ecf4de145ce48e98d5c049e41e1622d
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built bioinfokit adjustText


In [98]:

#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging
import tensorflow as tf
import seaborn as sns
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from bioinfokit.visuz import cluster


In [99]:


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)
#@title Compute a representation for each message, showing various lengths supported.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [100]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(Texts)
message_embeddings1=message_embeddings.numpy()
message_embeddings1

array([[ 0.0443922 , -0.04483264,  0.04483263, ..., -0.04483264,
        -0.04483264,  0.04483264],
       [ 0.04502835, -0.04507171,  0.04507103, ..., -0.04507171,
        -0.04507171,  0.04507171],
       [ 0.04505383, -0.04505383,  0.04137913, ..., -0.04505383,
        -0.04505383,  0.04505176],
       ...,
       [ 0.04547684, -0.04553784, -0.04540149, ..., -0.04553784,
        -0.04553784,  0.04553716],
       [ 0.04466214, -0.04466214,  0.03023672, ..., -0.04466214,
        -0.04466214,  0.04466214],
       [ 0.04475961, -0.04477896,  0.04477896, ..., -0.04477896,
        -0.04477896,  0.04477896]], dtype=float32)

In [101]:

from sklearn.cluster import KMeans
SEED = 42

kmeans = KMeans(n_clusters=3, random_state=SEED).fit(message_embeddings1)
base_recapitulative['Clusters']=kmeans.labels_

In [102]:
d=base_recapitulative.sort_values(by="Clusters")
d

,Discours,Ville,Coordonnées,Date,Clusters
2,Bonjour tous merci présence ici Calais Devant ...,Calais,,,0
4,Bonjour chers amis Bonjour merci merci mille f...,Chambery,,,0
5,chers amis mesdames messieurs élus Quelle bell...,Chateaudun,,,0
7,Chers amis bravé froid bravé pluie bravé vent ...,Montsaintmichel,,,0
15,Mesdames Messieurs Bonjour tous merci ’ être v...,Défense,,,0
9,Bonjour tous bonjour amis Quelle passion entho...,Saintquentin,,,0
10,Bonjour tous amis Merci merci cet accueil chal...,Saulieu,,,0
8,Bonjour tous Merci beaucoup merci… Décidément ...,Sables,,,1
14,Mesdames Messieurs journalistes premiers derni...,Presse,,,1
13,Bonsoir toutes Chères amies chères compatriote...,Femmes,,,2


In [103]:
  corr = np.inner(message_embeddings1, message_embeddings1)
  aa=sns.heatmap(corr,xticklabels=d.Ville,yticklabels=d.Ville)
  aa.set_title("Semantic Textual Similarity")
  


Text(0.5, 1.0, 'Semantic Textual Similarity')

In [104]:
d.Ville

2              Calais
4            Chambery
5          Chateaudun
7     Montsaintmichel
15            Défense
9        Saintquentin
10            Saulieu
8              Sables
14             Presse
13             Femmes
12             Toulon
0                Agen
6               Lille
3              Cannes
1          Villepinte
11            Sologne
16          Trocadero
Name: Ville, dtype: object

In [105]:
corr1=pd.DataFrame(corr,index=d.Ville,columns=d.Ville)
corr1
clus=sns.clustermap(corr1, cmap = 'viridis', method = 'ward')
!pip install pca

/usr/local/lib/python3.7/dist-packages/seaborn/matrix.py:805: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(figsize=figsize)


  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=eedf4dcfb1e26c94c1399c9e84102e4afcfd1a95c24960dbff221b6616fb7bd4
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [106]:
corr1

Ville,Calais,Chambery,Chateaudun,Montsaintmichel,Défense,Saintquentin,Saulieu,Sables,Presse,Femmes,Toulon,Agen,Lille,Cannes,Villepinte,Sologne,Trocadero
Ville,,,,,,,,,,,,,,,,,
Calais,0.999999,0.933536,0.736802,0.919264,0.830575,0.870493,0.939236,0.867261,0.797064,0.884710,0.894296,0.894062,0.903876,0.847022,0.840392,0.818027,0.887686
Chambery,0.933536,0.999997,0.747350,0.942778,0.853583,0.886696,0.891764,0.889700,0.844423,0.869725,0.870428,0.897667,0.896634,0.815151,0.871995,0.860706,0.897549
Chateaudun,0.736802,0.747350,0.999998,0.737874,0.771965,0.760458,0.714312,0.764451,0.797047,0.736003,0.750279,0.768295,0.767279,0.710367,0.746682,0.771394,0.738147
Montsaintmichel,0.919264,0.942778,0.737874,0.999999,0.865338,0.892995,0.901867,0.900820,0.812171,0.875157,0.904028,0.913363,0.910727,0.816734,0.828529,0.849390,0.918736
Défense,0.830575,0.853583,0.771965,0.865338,0.999999,0.866584,0.825340,0.885594,0.792009,0.858108,0.886560,0.846864,0.890079,0.778501,0.790518,0.879268,0.893921
Saintquentin,0.870493,0.886696,0.760458,0.892995,0.866584,1.000002,0.840350,0.911995,0.790910,0.865697,0.902545,0.887744,0.841566,0.754614,0.801678,0.876897,0.844732
Saulieu,0.939236,0.891764,0.714312,0.901867,0.825340,0.840350,0.999999,0.857862,0.753747,0.864193,0.883258,0.850156,0.887581,0.813543,0.781387,0.790313,0.877599
Sables,0.867261,0.889700,0.764451,0.900820,0.885594,0.911995,0.857862,1.000003,0.774546,0.894921,0.912417,0.856170,0.874321,0.758010,0.786248,0.912144,0.870775
Presse,0.797064,0.844423,0.797047,0.812171,0.792009,0.790910,0.753747,0.774546,0.999998,0.757710,0.755700,0.836784,0.802277,0.732546,0.873737,0.780414,0.790920


In [107]:

pca_out = PCA().fit(corr1)

In [108]:


# component loadings
loadings = pca_out.components_


# get eigenvalues (variance explained by each PC)  
pca_out.explained_variance_



array([3.1192126e-02, 1.1477010e-02, 7.5379540e-03, 3.4326667e-03,
       3.1419646e-03, 1.7654049e-03, 1.1003576e-03, 8.1870524e-04,
       6.4507627e-04, 5.9911283e-04, 3.9334910e-04, 3.0310533e-04,
       2.0885447e-04, 1.4439736e-04, 9.0358553e-05, 5.1995441e-05,
       2.9691430e-16], dtype=float32)

In [109]:

# get biplot
pca_scores = PCA().fit_transform(corr1)


In [136]:
cluster.biplot(cscore=pca_scores, loadings=loadings, labels=corr1.columns, var1=round(pca_out.explained_variance_ratio_[0]*100, 2),
    var2=round(pca_out.explained_variance_ratio_[1]*100, 2),dim=(10,8))

/usr/local/lib/python3.7/dist-packages/bioinfokit/visuz.py:1839: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.subplots(figsize=dim)


In [137]:
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(Texts[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Bonjour Agen Bonjour chers amis Bonjour Nouvelle Aquitaine plaisir plaisir ’ être retour Sud-Ouest Agen depuis enfant ’ Rugby ’ grande équipe gaillards fiers valeureux affrontent meilleures équipes France entière Agen ’ terre chaleureuse où fait bon vivre où beauté paysages ’ ’ égal richesse gastronomie Agen ’ ’ hospitalité chaleur Agenais Merci ’ accueillir ici chez chers amis heureux ’ être aujourd ’ hui aujourd ’ hui ’ grand jour 12 mars 2022 mois quasiment jour jour ’ isoloir ’ isoloir rendez-vous destin France rendez-vous ’ avenir enfants rendez-vous ’ Histoire pays ’ isoloir plusieurs bulletins posés devant vérité ’ deux mien celui tous autres choix entre France telle connaissons grand saut ’ inconnu ’ autres candidats proposent proposent saut vide proposent France nouvelle comme disent nouvelle française proposent « payez voir » proposent rien protéger risques viennent… chers amis comme politicien professionnel savez ’ abandonné vie menais rentrer campagne ’ accepté pre

In [112]:
aa=sns.heatmap(corr,xticklabels=base_recapitulative.Ville,yticklabels=base_recapitulative.Ville)
plt.show()
d=base_recapitulative.sort_values(by="Clusters")
d

/usr/local/lib/python3.7/dist-packages/seaborn/matrix.py:330: UserWarning: Use the colorbar set_ticks() method instead.
  ax.set(xticks=xticks, yticks=yticks)


,Discours,Ville,Coordonnées,Date,Clusters
2,Bonjour tous merci présence ici Calais Devant ...,Calais,,,0
4,Bonjour chers amis Bonjour merci merci mille f...,Chambery,,,0
5,chers amis mesdames messieurs élus Quelle bell...,Chateaudun,,,0
7,Chers amis bravé froid bravé pluie bravé vent ...,Montsaintmichel,,,0
15,Mesdames Messieurs Bonjour tous merci ’ être v...,Défense,,,0
9,Bonjour tous bonjour amis Quelle passion entho...,Saintquentin,,,0
10,Bonjour tous amis Merci merci cet accueil chal...,Saulieu,,,0
8,Bonjour tous Merci beaucoup merci… Décidément ...,Sables,,,1
14,Mesdames Messieurs journalistes premiers derni...,Presse,,,1
13,Bonsoir toutes Chères amies chères compatriote...,Femmes,,,2


# Ajout Dates+Coordonnées 

In [113]:
df=pd.DataFrame(base_recapitulative)

In [114]:
df

,Discours,Ville,Coordonnées,Date,Clusters
0,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,Agen,,,2
1,Bonjour tous bonjour amis… Merci accueil ’ inc...,Villepinte,,,2
2,Bonjour tous merci présence ici Calais Devant ...,Calais,,,0
3,Merci amis merci fond cœur presence joie encou...,Cannes,,,2
4,Bonjour chers amis Bonjour merci merci mille f...,Chambery,,,0
5,chers amis mesdames messieurs élus Quelle bell...,Chateaudun,,,0
6,Bonjour tous chers amis plaisir ’ être retour ...,Lille,,,2
7,Chers amis bravé froid bravé pluie bravé vent ...,Montsaintmichel,,,0
8,Bonjour tous Merci beaucoup merci… Décidément ...,Sables,,,1
9,Bonjour tous bonjour amis Quelle passion entho...,Saintquentin,,,0


In [115]:
vvv=['03/12/2022',
' 12/05/2021  ',
' 01/19/2022  ',
' 01/23/2022  ',
' 02/25/2022  ',
' 01/07/2022  ',
' 02/05/2022  ',
' 02/19/2022  ',
' 01/08/2022  ',
' 01/15/2022   ',
' 02/12/2022  ',
' 01/28/2022  ',
' 03/06/2022  ',
'03/08/2022',
'01/10/2022',
'02/17/2022',
'03/27/2022']

In [116]:
df['date']=vvv


In [117]:
## correcting the name of place as for two part name it is without space between two parts but the second part start with capital letter


df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['month'] = df['date'].dt.month
df['year']=df['date'].dt.year

In [118]:
df

,Discours,Ville,Coordonnées,Date,Clusters,date,day,weekday,month,year
0,Bonjour Agen Bonjour chers amis Bonjour Nouvel...,Agen,,,2,2022-03-12,12,5,3,2022
1,Bonjour tous bonjour amis… Merci accueil ’ inc...,Villepinte,,,2,2021-12-05,5,6,12,2021
2,Bonjour tous merci présence ici Calais Devant ...,Calais,,,0,2022-01-19,19,2,1,2022
3,Merci amis merci fond cœur presence joie encou...,Cannes,,,2,2022-01-23,23,6,1,2022
4,Bonjour chers amis Bonjour merci merci mille f...,Chambery,,,0,2022-02-25,25,4,2,2022
5,chers amis mesdames messieurs élus Quelle bell...,Chateaudun,,,0,2022-01-07,7,4,1,2022
6,Bonjour tous chers amis plaisir ’ être retour ...,Lille,,,2,2022-02-05,5,5,2,2022
7,Chers amis bravé froid bravé pluie bravé vent ...,Montsaintmichel,,,0,2022-02-19,19,5,2,2022
8,Bonjour tous Merci beaucoup merci… Décidément ...,Sables,,,1,2022-01-08,8,5,1,2022
9,Bonjour tous bonjour amis Quelle passion entho...,Saintquentin,,,0,2022-01-15,15,5,1,2022


In [119]:
from tkinter import *
from geopy.geocoders import Nominatim

In [120]:
geolocator = Nominatim(user_agent="MyApp")

In [121]:
!pip install pgeocode

In [122]:
import pgeocode

In [123]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 6.2 MB/s 
     |████████████████████████████████| 16.7 MB 36.9 MB/s 
     |████████████████████████████████| 6.3 MB 51.1 MB/s 


In [124]:
nomi = pgeocode.Nominatim('fr')

In [125]:
print(nomi.query_location("London"))

None


In [126]:
nomi.query_postal_code("75013")

postal_code                 75013
country_code                   FR
place_name        Paris 13, Paris
state_name          Île-de-France
state_code                   11.0
county_name                 Paris
county_code                    75
community_name              Paris
community_code                751
latitude                  48.8322
longitude                 2.35245
accuracy                        5
Name: 0, dtype: object

In [127]:
list_ville=base_recapitulative.Ville.drop(labels=[13,14,15])
list_ville1=list_ville

In [128]:
list_ville[7]="Mont-Saint-Michel"

In [129]:
list_ville[8]="Les Sables-d’Olonne"

In [130]:
list_ville[9]="Saint-Quentin"

In [131]:
list_ville[11]="Chaumont-sur-Tharonne"

In [132]:
list_ville[13]="Paris"

In [133]:
kp=[]
kp1=[]

for i in list_ville:
  print(i)
  location = geolocator.geocode(i)
  kp.append(location.longitude)
  kp1.append(location.latitude)


Agen
Villepinte
Calais
Cannes
Chambery
Chateaudun
Lille
Mont-Saint-Michel
Les Sables-d’Olonne
Saint-Quentin
Saulieu
Chaumont-sur-Tharonne
Toulon
Trocadero
Paris


In [134]:
df2=pd.DataFrame()
df2["Ville"]=list_ville1
df2["longitude"]=kp
df2["latitude"]=kp1

df3=df.merge(df2,how="left",on="Ville")